# Utilizando GeoPy

In [1]:
from geopy.geocoders import Nominatim

In [2]:
geolocator = Nominatim(user_agent="primeiro_teste_com_Nominatim", timeout=10)

In [3]:
location = geolocator.geocode("Rua São Bento, 407")

In [4]:
location

Location(Rua São Bento, Vila Alto do Cruzeiro, Praia Grande, Salvador, Microrregião de Salvador, Região Metropolitana de Salvador, Região Geográfica Intermediária de Salvador, Bahia, Região Nordeste, 40725-610, Brasil, (-12.8732009, -38.4717929, 0.0))

In [5]:
geocode = lambda query: geolocator.geocode("%s, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste" % query)

In [6]:
location = geocode("rua são bento, 405")

In [7]:
location

Location(Rua São Bento, Quitaúna, Osasco, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 06186-230, Brasil, (-23.5338883, -46.8164347, 0.0))

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv(f"dados/exemplo-1.csv")

In [10]:
df

,id,endereço
0,1,TESTE
1,2,"rua da entidade,25"
2,3,Av. Augusto Antunes 337
3,5,Ermelino Matarazzo e comunidade vila Silvia
4,6,Rua Serra de LUIZ Gomes 1992 cep 08190 500
...,...,...
778,855,"Rua Maria Amália Lopes de Azevedo, 4.180. Jd. ..."
779,856,Projeto fazendo a diferença.. Centro de São Pa...
780,857,RUA ITAPIRU 137
781,858,Rua do outono 410


In [11]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [12]:
from tqdm import tqdm
tqdm.pandas()

/home/fernando/miniconda3/envs/geo_env/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
df['location'] = df['endereço'].progress_apply(geocode)

100%|██████████| 783/783 [24:54<00:00,  1.91s/it]


In [14]:
df

,id,endereço,location
0,1,TESTE,"(Teste, Otacílio Costa, Região Geográfica Imed..."
1,2,"rua da entidade,25",None
2,3,Av. Augusto Antunes 337,"(Avenida Augusto Antunes, Jardim São Carlos, L..."
3,5,Ermelino Matarazzo e comunidade vila Silvia,None
4,6,Rua Serra de LUIZ Gomes 1992 cep 08190 500,None
...,...,...,...
778,855,"Rua Maria Amália Lopes de Azevedo, 4.180. Jd. ...",None
779,856,Projeto fazendo a diferença.. Centro de São Pa...,None
780,857,RUA ITAPIRU 137,"(137, Rua Itapiru, Vila da Saúde, Saúde, São P..."
781,858,Rua do outono 410,"(Rua do Outono, Brasilândia, São Paulo, Região..."


In [15]:
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [16]:
df[~df.point.isin([None])]

,id,endereço,location,point
0,1,TESTE,"(Teste, Otacílio Costa, Região Geográfica Imed...","(-27.5106495, -50.1162173, 0.0)"
2,3,Av. Augusto Antunes 337,"(Avenida Augusto Antunes, Jardim São Carlos, L...","(-23.5122828, -46.4686618, 0.0)"
6,8,Av engenheiro pinto 318,"(Praça Xavier Pinto Lima, Engenheiro Goulart, ...","(-23.4998295, -46.5243336, 0.0)"
7,9,Rua Lucas Nogueira Garcez 21,"(Rua Lucas Nogueira Garcez, Jardim Baronesa, V...","(-23.4952509, -47.4906554, 0.0)"
9,11,Rua Júlio cenalvo 16a,"(Rua Júlio Cenalvo, Aricanduva, São Paulo, Reg...","(-23.5828046, -46.5147215, 0.0)"
...,...,...,...,...
773,850,Av. Miguel conejo 20 freguesia do Ô São Paulo,"(Avenida Miguel Conejo, Parque Monteiro Soares...","(-23.4968875, -46.69289, 0.0)"
774,851,Rua Alcidio Gomes N 103,"(Rua Alcídio Gomes, Lagoa Grande, Parelheiros,...","(-23.866867, -46.7560378, 0.0)"
777,854,Rua alexandre groppali 215,"(Rua Alexandre Groppali, Fazenda da Juta, Jard...","(-23.613712, -46.488996, 0.0)"
780,857,RUA ITAPIRU 137,"(137, Rua Itapiru, Vila da Saúde, Saúde, São P...","(-23.6135668, -46.6365465, 0.0)"


In [19]:
df.to_csv("resultados/exemplo-1-nominatim.csv")

In [21]:
import geopandas as gpd

In [22]:
df_points = df[~df.point.isin([None])]

In [37]:
df_points['x'] = df_points.apply(lambda row: row.point[1], axis=1)
df_points['y'] = df_points.apply(lambda row: row.point[0], axis=1)

<ipython-input-37-a89875fe4032>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_points['x'] = df_points.apply(lambda row: row.point[1], axis=1)
<ipython-input-37-a89875fe4032>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_points['y'] = df_points.apply(lambda row: row.point[0], axis=1)


In [38]:
gdf = gpd.GeoDataFrame(df_points, geometry=gpd.points_from_xy(df_points.x, df_points.y))
# df_points.point[0][0]

In [39]:
gdf

,id,endereço,location,point,x,y,geometry
0,1,TESTE,"(Teste, Otacílio Costa, Região Geográfica Imed...","(-27.5106495, -50.1162173, 0.0)",-50.116217,-27.510649,POINT (-50.11622 -27.51065)
2,3,Av. Augusto Antunes 337,"(Avenida Augusto Antunes, Jardim São Carlos, L...","(-23.5122828, -46.4686618, 0.0)",-46.468662,-23.512283,POINT (-46.46866 -23.51228)
6,8,Av engenheiro pinto 318,"(Praça Xavier Pinto Lima, Engenheiro Goulart, ...","(-23.4998295, -46.5243336, 0.0)",-46.524334,-23.499830,POINT (-46.52433 -23.49983)
7,9,Rua Lucas Nogueira Garcez 21,"(Rua Lucas Nogueira Garcez, Jardim Baronesa, V...","(-23.4952509, -47.4906554, 0.0)",-47.490655,-23.495251,POINT (-47.49066 -23.49525)
9,11,Rua Júlio cenalvo 16a,"(Rua Júlio Cenalvo, Aricanduva, São Paulo, Reg...","(-23.5828046, -46.5147215, 0.0)",-46.514722,-23.582805,POINT (-46.51472 -23.58280)
...,...,...,...,...,...,...,...
773,850,Av. Miguel conejo 20 freguesia do Ô São Paulo,"(Avenida Miguel Conejo, Parque Monteiro Soares...","(-23.4968875, -46.69289, 0.0)",-46.692890,-23.496887,POINT (-46.69289 -23.49689)
774,851,Rua Alcidio Gomes N 103,"(Rua Alcídio Gomes, Lagoa Grande, Parelheiros,...","(-23.866867, -46.7560378, 0.0)",-46.756038,-23.866867,POINT (-46.75604 -23.86687)
777,854,Rua alexandre groppali 215,"(Rua Alexandre Groppali, Fazenda da Juta, Jard...","(-23.613712, -46.488996, 0.0)",-46.488996,-23.613712,POINT (-46.48900 -23.61371)
780,857,RUA ITAPIRU 137,"(137, Rua Itapiru, Vila da Saúde, Saúde, São P...","(-23.6135668, -46.6365465, 0.0)",-46.636547,-23.613567,POINT (-46.63655 -23.61357)


In [49]:
# gdf.to_file('resultados/exemplo-1-nominatim.gpkg')
gdf[['id', 'endereço', 'geometry']].to_file('resultados/exemplo-1-nominatim.gpkg', driver="GPKG")